In [78]:
from keras import backend as K
import time
import matplotlib.pyplot as plt
import numpy as np_utils
%matplotlib inline
np.random.seed(2017)
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, DepthwiseConv2D, Conv2D, SeparableConv2D, MaxPooling1D
from keras.layers import Input, concatenate
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import GlobalAveragePooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Nadam, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.regularizers import l2
%config InlineBackend.figure_format='retina'
from keras_contrib.callbacks import CyclicLR
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from data_science_utils.vision.keras import *
from time import time
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import missingno as msno
import re
from joblib import Parallel, delayed
from data_science_utils import dataframe as df_utils
from data_science_utils import models as model_utils
from data_science_utils import plots as plot_utils
from data_science_utils.dataframe import column as column_utils
from data_science_utils import misc as misc
from data_science_utils import preprocessing as pp_utils
from data_science_utils import nlp as nlp_utils

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from data_science_utils.dataframe import get_specific_cols

import more_itertools
from more_itertools import flatten
import ast
from sklearn.preprocessing import LabelEncoder

In [4]:
df_train = pd.read_csv("price_prediction/train.csv")
df_test = pd.read_csv("price_prediction/test.csv")

In [ ]:
df_train.head()
df_test.head()


# Baseline
- GL Mean
- GL Median

In [15]:
df_gl_means = df_train.groupby(["GL"])[['PRICE']].mean().reset_index()

df_results = df_test.merge(df_gl_means, on=["GL"],how="left")
df_results = df_results[["ID","PRICE"]]
df_results["PRICE"] = df_results["PRICE"].fillna(df_results["PRICE"].mean())

df_results.head()

,ID,PRICE
0,1585751,772.612384
1,1530678,1290.724354
2,1324955,772.612384
3,822454,2321.772340
4,1604015,1115.235239


In [16]:
df_results.to_csv("baseline.csv",index=False)


## For Word Embedding CNN
- Making a text column
    - append GL
    - replace num
    - replace measurement
    
- Hyper Params here:
    - Stopwords
    - word_length_filter
    - lemmatize or not
    - vocab_size in build_dict
    
    
    


# Baseline 2: Keras Imdb Model

In [ ]:
from __future__ import print_function


# set parameters:
max_features = 50000
maxlen = 100
batch_size = 256
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 5

print('Loading data...')

X,y = df_train['text_encoded'].values,df_train['PRICE'].values
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mae','mse'])
model.count_params()

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))



1159686/1159686 [==============================] - 341s 294us/step - loss: 4033933786.3938 - mean_absolute_error: 894.5309 - val_loss: 16347761.4813 - val_mean_absolute_error: 716.5794
Epoch 2/5
1159686/1159686 [==============================] - 340s 293us/step - loss: 4032204864.3942 - mean_absolute_error: 822.3543 - val_loss: 15832785.5048 - val_mean_absolute_error: 704.4332
Epoch 3/5
1159686/1159686 [==============================] - 339s 292us/step - loss: 4031421547.0617 - mean_absolute_error: 794.2296 - val_loss: 15649164.7560 - val_mean_absolute_error: 698.5270
Epoch 4/5
1159686/1159686 [==============================] - 341s 294us/step - loss: 4030308633.8232 - mean_absolute_error: 781.7042 - val_loss: 15564212.7209 - val_mean_absolute_error: 678.7286
Epoch 5/5
1159686/1159686 [==============================] - 342s 295us/step - loss: 4028990347.3919 - mean_absolute_error: 773.1664 - val_loss: 15462277.1944 - val_mean_absolute_error: 689.7498


In [ ]:
X_test = df_test['text_encoded'].values
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
y_preds = model.predict(X_test)

df_results = df_test[['ID']]
df_results['PRICE'] = y_preds

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [63]:
df_results.head()
df_results.to_csv("baseline-2.csv",index=False)

,ID,PRICE
0,1585751,402.462646
1,1530678,708.144775
2,1324955,199.348129
3,822454,2087.896729
4,1604015,637.491211


In [61]:
model.count_params()

model.summary()

2600751

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 50)           2500000   
_________________________________________________________________
dropout_7 (Dropout)          (None, 100, 50)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 98, 250)           37750     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 250)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_8 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_4 (Activation)    (None, 250)               0         
__________

# Embedding Model

In [ ]:
max_features = 50000
maxlen = 100
batch_size = 4096
embedding_dims = 50
epochs = 5


X,y = df_train['text_encoded'].values,df_train['PRICE'].values
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

inputs = Input(shape=(maxlen,), dtype='int32', name='main_input')
x = Embedding(output_dim=embedding_dims, input_dim=max_features, input_length=maxlen)(inputs)
K.int_shape(x)
x = conv_layer(x,n_kernels=64,kernel_size=5,padding='valid')
x = conv_layer(x,n_kernels=128,kernel_size=3,padding='valid')
x = transition_layer(x, n_kernels=64,dropout=0)
x = MaxPooling1D()(x)
x = conv_layer(x,n_kernels=128,kernel_size=3,padding='valid')
x = conv_layer(x,n_kernels=256,kernel_size=3,padding='valid')
x = transition_layer(x, n_kernels=64,dropout=0)



xp = pre_dense_layer(x)
K.int_shape(x)
print(K.int_shape(xp))
x = Dense(128)(xp)
x = Dropout(0.2)(x)
x = Activation('relu')(x)
K.int_shape(x)
output = Dense(1)(x)

model = Model(inputs=inputs, outputs=output)


model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mae','mse'])
print(model.count_params())

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))



# Pretrained Embeddings Model

In [ ]:
maxlen = 100
batch_size = 4096
embedding_dims = 300
epochs = 5


X,y = ptr.transform(df_train['text'].values),df_train['PRICE'].values
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
# x_test = sequence.pad_sequences(x_test, maxlen=maxlen)


Fasttext Transforms start at: 2019-06-12 15:50:49.517983
Number of Unique Test Tokens for Fasttext transform 332242


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:20: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:21: RuntimeWarning: invalid value encountered in true_divide


In [113]:

inputs = Input(shape=(None,embedding_dims), dtype='int32', name='main_input')

x = conv_layer(x,n_kernels=64,kernel_size=5,padding='valid')
x = conv_layer(x,n_kernels=128,kernel_size=3,padding='valid')
x = transition_layer(x, n_kernels=64,dropout=0)
x = MaxPooling1D()(x)
x = conv_layer(x,n_kernels=128,kernel_size=3,padding='valid')
x = conv_layer(x,n_kernels=256,kernel_size=3,padding='valid')
x = transition_layer(x, n_kernels=64,dropout=0)


xp = pre_dense_layer(x)
x = Dense(128)(xp)
x = Dropout(0.2)(x)
x = Activation('relu')(x)
K.int_shape(x)
output = Dense(1)(x)

model = Model(inputs=inputs, outputs=output)


model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mae','mse'])
print(model.count_params())

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))



ValueError: Input 0 is incompatible with layer conv1d_44: expected ndim=3, found ndim=2

# Post Processing
- gl min max clipping
- max(0,output)